To create a new table we'd still use the Table object; however, we'd need to replace the autoload and autoload_with parameters with Column objects.

The Column object takes a name, a SQLAlchemy type with an optional format, and optional keyword arguments for different constraints. 

In [2]:
# Import Table, Column, String, Integer, Float, Boolean from sqlalchemy
from sqlalchemy import Table, Column, String, Integer , Float , Boolean,create_engine
from sqlalchemy import MetaData
metadata=MetaData()
engine=create_engine('sqlite:///mydata.sqlite')
# Define a new table with a name, count, amount, and valid column: data
data = Table('data',metadata,
             Column('name', String(255)),
             Column('count', Integer()),
             Column('amount', Float()),
             Column('valid', Boolean())
)

# Use the metadata to create the table
metadata.create_all(engine)

# Print table details
print(repr(data))


Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>), schema=None)


In [11]:
# Import Table, Column, String, Integer, Float, Boolean from sqlalchemy
from sqlalchemy import Table, Column, String, Integer, Float, Boolean

# Define a new table with a name, count, amount, and valid column: data
data = Table('data_table1', metadata,
             Column('name', String(255), unique=True),
             Column('count', Integer(), default=1),
             Column('amount', Float()),
             Column('valid', Boolean(), default=False)
)

# Use the metadata to create the table
metadata.create_all(engine)





Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>), schema=None)


## Inserting a single row with an insert() statement

There are several ways to perform an insert with SQLAlchemy; however, we are going to focus on the one that follows the same pattern as the select statement. It uses an insert statement where we specify the table as an argument, and supply the data we wish to insert into the value via the .values() method as keyword arguments.


In [13]:
# Import insert and select from sqlalchemy
from sqlalchemy import insert, select
data_table1= Table('data_table1',metadata, autoload=True, autoload_with=engine)
# Build an insert statement to insert a record into the data table: stmt
stmt = insert(data_table1).values(name='Anna', count=1, amount=1000.00, valid=True)

# Execute the statement via the connection: results
results = connection.execute(stmt)

# Print result rowcount
print(results.rowcount)

# Build a select statement to validate the insert
stmt = select([data]).where(data.columns.name =='Anna')

# Print the result of executing the query.
print(connection.execute(stmt).first())


1
('Anna', 1, 1000.0, True)


## Inserting Multiple Records at Once

We'll want to first build a list of dictionaries that represents the data we want to insert. Then, in the .execute() method, we can pair this list of dictionaries with an insert statement, which will insert all the records in our list of dictionaries.

In [15]:
# Build a list of dictionaries: values_list
values_list = [
    {'name':'Subham', 'count':1, 'amount':1000.00, 'valid':True},
    {'name':'Taylor', 'count':1, 'amount':750.00, 'valid':False}
]

# Build an insert statement for the data table: stmt
stmt =insert(data)

# Execute stmt with the values_list: results
results = connection.execute(stmt,values_list)

# Print rowcount
print(results.rowcount)


2


## Loading a CSV into a Table

In [16]:
# Create a insert statement for census: stmt
stmt = insert(census)

# Create an empty list and zeroed row count: values_list, total_rowcount
values_list = []
total_rowcount = 0

# Enumerate the rows of csv_reader
for idx, row in enumerate(csv_reader):
    #create data and append to values_list
    data = {'state': row[0], 'sex': row[1], 'age': row[2], 'pop2000': row[3],
            'pop2008': row[4]}
    values_list.append(data)

    # Check to see if divisible by 51
    if idx % 51 == 0:
        results = connection.execute(stmt, values_list)
        total_rowcount += results.rowcount
        values_list = []

# Print total rowcount
print(total_rowcount)


NameError: name 'census' is not defined

## Updating individual records

The update statement is very similar to an insert statement, except that it also typically uses a where clause to help us determine what data to update.

In [25]:
engine =create_engine('sqlite:///census.sqlite')
state_fact = Table('state_fact',metadata, autoload=True, autoload_with=engine)
print(state_fact.columns)
# Build a select statement: select_stmt
select_stmt = select([state_fact]).where(state_fact.columns.name == 'New York')

# Print the results of executing the select_stmt
print(connection.execute(select_stmt).fetchall())

# Build a statement to update the fips_state to 36: stmt
stmt = update(state_fact).values(fips_state=36)

# Append a where clause to limit it to records for New York state
stmt = stmt.where(state_fact.columns.name == 'New York')

# Execute the statement: results
results = connection.execute(stmt)

# Print rowcount
print(results.rowcount)

# Execute the select_stmt again to view the changes
print(connection.execute(select_stmt).fetchall())


['state_fact.id', 'state_fact.name', 'state_fact.abbreviation', 'state_fact.country', 'state_fact.type', 'state_fact.sort', 'state_fact.status', 'state_fact.occupied', 'state_fact.notes', 'state_fact.fips_state', 'state_fact.assoc_press', 'state_fact.standard_federal_region', 'state_fact.census_region', 'state_fact.census_region_name', 'state_fact.census_division', 'state_fact.census_division_name', 'state_fact.circuit_court']


OperationalError: (sqlite3.OperationalError) no such table: state_fact [SQL: 'SELECT state_fact.id, state_fact.name, state_fact.abbreviation, state_fact.country, state_fact.type, state_fact.sort, state_fact.status, state_fact.occupied, state_fact.notes, state_fact.fips_state, state_fact.assoc_press, state_fact.standard_federal_region, state_fact.census_region, state_fact.census_region_name, state_fact.census_division, state_fact.census_division_name, state_fact.circuit_court \nFROM state_fact \nWHERE state_fact.name = ?'] [parameters: ('New York',)]

## Updating Multiple Records



In [27]:
from sqlalchemy import update
# Build a statement to update the notes to 'The Wild West': stmt
stmt = update(state_fact).values(notes='The Wild West')

# Append a where clause to match the West census region records
stmt = stmt.where(state_fact.columns.census_region_name=='West')

# Execute the statement: results
results = connection.execute(stmt)

# Print rowcount
print(results.rowcount)


OperationalError: (sqlite3.OperationalError) no such table: state_fact [SQL: 'UPDATE state_fact SET notes=? WHERE state_fact.census_region_name = ?'] [parameters: ('The Wild West', 'West')]

## Correlated Updates

In [28]:
# Build a statement to select name from state_fact: stmt
fips_stmt = select([state_fact.columns.name])

# Append a where clause to Match the fips_state to flat_census fips_code
fips_stmt = fips_stmt.where(
    state_fact.columns.fips_state == flat_census.columns.fips_code)

# Build an update statement to set the name to fips_stmt: update_stmt
update_stmt = update(flat_census).values(state_name=fips_stmt)

# Execute update_stmt: results
results = connection.execute(update_stmt)

# Print rowcount
print(results.rowcount)


NameError: name 'flat_census' is not defined

## Deleting all the records from a table

In [29]:
# Import delete, select
from sqlalchemy import select, delete
census = Table('census',metadata, autoload=True, autoload_with=engine)
# Build a statement to empty the census table: stmt
stmt = delete(census)

# Execute the statement: results
results = engine.execute(stmt)

# Print affected rowcount
print(results.rowcount)

# Build a statement to select all records from the census table
stmt = select([census])

# Print the results of executing the statement to verify there are no rows
print(connection.execute(stmt).fetchall())


8772


OperationalError: (sqlite3.OperationalError) no such table: census [SQL: 'SELECT census.state, census.sex, census.age, census.pop2000, census.pop2008 \nFROM census']

##  Deleting specific records

By using a where() clause, we can target the delete statement to remove only certain records. 

In [32]:
from sqlalchemy import func,and_
# Build a statement to count records using the sex column for Men ('M') age 36: stmt
stmt = select([func.count(census.columns.sex)]).where(
    and_(census.columns.sex == 'M',
         census.columns.age == 36)
)

# Execute the select statement and use the scalar() fetch method to save the record count
to_delete = connection.execute(stmt).scalar()

# Build a statement to delete records from the census table: stmt_del
stmt_del = delete(census)

# Append a where clause to target Men ('M') age 36
stmt_del = stmt_del.where(
    and_(census.columns.sex =='M',
         census.columns.age == 36)
)

# Execute the statement: results
results = connection.execute(stmt_del)

# Print affected rowcount and to_delete record count, make sure they match
print(results.rowcount, to_delete)


OperationalError: (sqlite3.OperationalError) no such table: census [SQL: 'SELECT count(census.sex) AS count_1 \nFROM census \nWHERE census.sex = ? AND census.age = ?'] [parameters: ('M', 36)]

## Deleting a Table Completely

In [33]:
# Drop the state_fact table

state_fact.drop(engine)
# Check to see if state_fact exists
print(state_fact.exists(engine))

# Drop all tables

metadata.drop_all(engine)
# Check to see if census exists
print(census.exists(engine))


False
False
